In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
num_epochs = 100
learning_rate=0.001

In [0]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])


In [5]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [0]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [0]:
# 5x5 convolution
def conv5x5(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=5, 
                     stride=stride, padding=2, bias=False)

In [0]:
# 7x7 convolution
def conv7x7(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=7, 
                     stride=stride, padding=3, bias=False)

In [0]:
# 1x1 convolution
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                     stride=stride, padding=0, bias=False)

In [0]:
class IdentityBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.elu(out)
        return out

In [0]:

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3, 64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.conv2 = conv5x5(128,128)
        self.bn2 = nn.BatchNorm2d(128)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.conv3 = conv3x3(512,512)
        self.bn3 = nn.BatchNorm2d(512)
#         self.fc1 = nn.Linear(,64)
#         self.fc2 = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = F.elu(self.bn2(self.conv2(out)))
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = F.relu(self.bn3(self.conv3(out)))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
#         out = self.fc2(64,10)
        return out


In [0]:
model=ResNet(IdentityBlock, [2,2,2,2]).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [0]:

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate

In [17]:
total_step

500

In [18]:
epoch_accuracy=0
for epoch in range(15):
    for i, (images, labels) in enumerate(train_loader):
        total=0
        correct=0
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #calculating accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Accuracy: {}%"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),(100 * correct / total)))
        
        epoch_accuracy+=correct / total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
    print("Accuracy in the epoch = {}%".format(epoch_accuracy/5))
    epoch_accuracy=0

# Test the model

Epoch [1/100], Step [100/500] Loss: 1.9303 Accuracy: 28.0%
Epoch [1/100], Step [200/500] Loss: 1.8685 Accuracy: 28.0%
Epoch [1/100], Step [300/500] Loss: 1.5056 Accuracy: 40.0%
Epoch [1/100], Step [400/500] Loss: 1.5325 Accuracy: 47.0%
Epoch [1/100], Step [500/500] Loss: 1.4958 Accuracy: 44.0%
Accuracy in the epoch = 38.601999999999975%
Epoch [2/100], Step [100/500] Loss: 1.3403 Accuracy: 44.0%
Epoch [2/100], Step [200/500] Loss: 1.1953 Accuracy: 49.0%
Epoch [2/100], Step [300/500] Loss: 1.1968 Accuracy: 60.0%
Epoch [2/100], Step [400/500] Loss: 1.3437 Accuracy: 51.0%
Epoch [2/100], Step [500/500] Loss: 1.1948 Accuracy: 54.0%
Accuracy in the epoch = 52.974000000000025%
Epoch [3/100], Step [100/500] Loss: 1.1761 Accuracy: 59.0%
Epoch [3/100], Step [200/500] Loss: 1.0781 Accuracy: 64.0%
Epoch [3/100], Step [300/500] Loss: 1.1500 Accuracy: 59.0%
Epoch [3/100], Step [400/500] Loss: 1.1256 Accuracy: 56.0%
Epoch [3/100], Step [500/500] Loss: 1.1499 Accuracy: 64.0%
Accuracy in the epoch = 59.

In [19]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 81.96 %


In [21]:
epoch_accuracy=0
for epoch in range(16,31):
    for i, (images, labels) in enumerate(train_loader):
        total=0
        correct=0
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #calculating accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Accuracy: {}%"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),(100 * correct / total)))
        
        epoch_accuracy+=correct / total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
    print("Accuracy in the epoch = {}%".format(epoch_accuracy/5))
    epoch_accuracy=0

# Test the model

Epoch [17/100], Step [100/500] Loss: 0.4272 Accuracy: 85.0%
Epoch [17/100], Step [200/500] Loss: 0.3930 Accuracy: 88.0%
Epoch [17/100], Step [300/500] Loss: 0.2811 Accuracy: 90.0%
Epoch [17/100], Step [400/500] Loss: 0.2562 Accuracy: 90.0%
Epoch [17/100], Step [500/500] Loss: 0.4963 Accuracy: 82.0%
Accuracy in the epoch = 86.33199999999992%
Epoch [18/100], Step [100/500] Loss: 0.3650 Accuracy: 88.0%
Epoch [18/100], Step [200/500] Loss: 0.3765 Accuracy: 88.0%
Epoch [18/100], Step [300/500] Loss: 0.4220 Accuracy: 84.0%
Epoch [18/100], Step [400/500] Loss: 0.2821 Accuracy: 89.0%
Epoch [18/100], Step [500/500] Loss: 0.3259 Accuracy: 93.0%
Accuracy in the epoch = 87.064%
Epoch [19/100], Step [100/500] Loss: 0.2751 Accuracy: 86.0%
Epoch [19/100], Step [200/500] Loss: 0.2332 Accuracy: 91.0%
Epoch [19/100], Step [300/500] Loss: 0.5069 Accuracy: 85.0%
Epoch [19/100], Step [400/500] Loss: 0.3902 Accuracy: 85.0%
Epoch [19/100], Step [500/500] Loss: 0.4926 Accuracy: 82.0%
Accuracy in the epoch = 8

In [22]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 88.39 %


In [23]:
epoch_accuracy=0
for epoch in range(31,50):
    for i, (images, labels) in enumerate(train_loader):
        total=0
        correct=0
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #calculating accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Accuracy: {}%"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),(100 * correct / total)))
        
        epoch_accuracy+=correct / total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 2
        update_lr(optimizer, curr_lr)
    print("Accuracy in the epoch = {}%".format(epoch_accuracy/5))
    epoch_accuracy=0

# Test the model

Epoch [32/100], Step [100/500] Loss: 0.0936 Accuracy: 98.0%
Epoch [32/100], Step [200/500] Loss: 0.1730 Accuracy: 93.0%
Epoch [32/100], Step [300/500] Loss: 0.1404 Accuracy: 95.0%
Epoch [32/100], Step [400/500] Loss: 0.2533 Accuracy: 89.0%
Epoch [32/100], Step [500/500] Loss: 0.1543 Accuracy: 95.0%
Accuracy in the epoch = 94.168%
Epoch [33/100], Step [100/500] Loss: 0.0738 Accuracy: 98.0%
Epoch [33/100], Step [200/500] Loss: 0.2959 Accuracy: 90.0%
Epoch [33/100], Step [300/500] Loss: 0.1771 Accuracy: 94.0%
Epoch [33/100], Step [400/500] Loss: 0.2879 Accuracy: 93.0%
Epoch [33/100], Step [500/500] Loss: 0.1436 Accuracy: 97.0%
Accuracy in the epoch = 94.42199999999994%
Epoch [34/100], Step [100/500] Loss: 0.1400 Accuracy: 96.0%
Epoch [34/100], Step [200/500] Loss: 0.1586 Accuracy: 93.0%
Epoch [34/100], Step [300/500] Loss: 0.1987 Accuracy: 90.0%
Epoch [34/100], Step [400/500] Loss: 0.1274 Accuracy: 94.0%
Epoch [34/100], Step [500/500] Loss: 0.1287 Accuracy: 96.0%
Accuracy in the epoch = 9

In [24]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 88.89 %


In [25]:
epoch_accuracy=0
for epoch in range(50,75):
    for i, (images, labels) in enumerate(train_loader):
        total=0
        correct=0
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #calculating accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Accuracy: {}%"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),(100 * correct / total)))
        
        epoch_accuracy+=correct / total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 2
        update_lr(optimizer, curr_lr)
    print("Accuracy in the epoch = {}%".format(epoch_accuracy/5))
    epoch_accuracy=0

# Test the model

Epoch [51/100], Step [100/500] Loss: 0.0355 Accuracy: 99.0%
Epoch [51/100], Step [200/500] Loss: 0.0505 Accuracy: 98.0%
Epoch [51/100], Step [300/500] Loss: 0.0879 Accuracy: 96.0%
Epoch [51/100], Step [400/500] Loss: 0.0586 Accuracy: 97.0%
Epoch [51/100], Step [500/500] Loss: 0.0737 Accuracy: 98.0%
Accuracy in the epoch = 97.64600000000034%
Epoch [52/100], Step [100/500] Loss: 0.0849 Accuracy: 98.0%
Epoch [52/100], Step [200/500] Loss: 0.0436 Accuracy: 100.0%
Epoch [52/100], Step [300/500] Loss: 0.1071 Accuracy: 97.0%
Epoch [52/100], Step [400/500] Loss: 0.0806 Accuracy: 98.0%
Epoch [52/100], Step [500/500] Loss: 0.1200 Accuracy: 95.0%
Accuracy in the epoch = 97.83600000000044%
Epoch [53/100], Step [100/500] Loss: 0.0267 Accuracy: 100.0%
Epoch [53/100], Step [200/500] Loss: 0.0974 Accuracy: 97.0%
Epoch [53/100], Step [300/500] Loss: 0.0289 Accuracy: 99.0%
Epoch [53/100], Step [400/500] Loss: 0.0801 Accuracy: 96.0%
Epoch [53/100], Step [500/500] Loss: 0.1045 Accuracy: 95.0%
Accuracy in 

In [26]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 89.58 %


In [29]:
epoch_accuracy=0
for epoch in range(75,100):
    for i, (images, labels) in enumerate(train_loader):
        total=0
        correct=0
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #calculating accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Accuracy: {}%"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),(100 * correct / total)))
        
        epoch_accuracy+=correct / total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 2
        update_lr(optimizer, curr_lr)
    print("Accuracy in the epoch = {}%".format(epoch_accuracy/5))
    epoch_accuracy=0

# Test the model

Epoch [76/100], Step [100/500] Loss: 0.0121 Accuracy: 100.0%
Epoch [76/100], Step [200/500] Loss: 0.0296 Accuracy: 100.0%
Epoch [76/100], Step [300/500] Loss: 0.0298 Accuracy: 98.0%
Epoch [76/100], Step [400/500] Loss: 0.0235 Accuracy: 100.0%
Epoch [76/100], Step [500/500] Loss: 0.0129 Accuracy: 100.0%
Accuracy in the epoch = 99.51400000000027%


KeyboardInterrupt: ignored

In [30]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 89.73 %


In [31]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): IdentityBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): IdentityBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')